In [1]:
import pandas as pd
import numpy as np

from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

KEY = "PKS7I0GX04CUGYTZ4ND7"
SECRET_KEY = "coYG5T8j3DvnJ9drddFiz3TxvAdFaqItEKw4qKYr"

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

### Check positions

In [17]:
positions = trading_client.get_all_positions()
sers = [pd.Series(dict(x)) for x in positions]
df = pd.concat(sers, axis=1)
df.T


### Get quotes for tickers

In [29]:
df = pd.read_csv("files/first_portfolio.csv")
ticks = df.ticker.to_list()

params = StockLatestQuoteRequest(symbol_or_symbols=ticks)
quotes = data_client.get_stock_latest_quote(params)

sers = [pd.Series(dict(x)) for x in quotes.values()]
df = pd.concat(sers, axis=1)
df.columns = [x for x in quotes]
df = df.T
df = df.drop(columns=["symbol"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113 entries, PINE to LZB
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   timestamp     113 non-null    datetime64[ns, UTC]
 1   ask_exchange  113 non-null    object             
 2   ask_price     113 non-null    object             
 3   ask_size      113 non-null    object             
 4   bid_exchange  113 non-null    object             
 5   bid_price     113 non-null    object             
 6   bid_size      113 non-null    object             
 7   conditions    113 non-null    object             
 8   tape          113 non-null    object             
dtypes: datetime64[ns, UTC](1), object(8)
memory usage: 8.8+ KB


### Compute desired trades

In [32]:
equity = 100000
numstocks = len(ticks)
dollars_per_stock = equity / numstocks

df.ask_price = np.where(df.ask_price==0, np.nan, df.ask_price)
df["trade"] = (dollars_per_stock / df.ask_price).astype(int)
ticks = df[df.trade>0].index.to_list()

### Submit trades

In [ ]:
for tick in ticks: 
    market_order_data = MarketOrderRequest(
        symbol=tick,
        qty=df.trade.loc[tick],
        side=OrderSide.BUY,
        time_in_force=TimeInForce.DAY
    )
    market_order = trading_client.submit_order(
        order_data=market_order_data
    )

### Check positions again

In [28]:
positions = trading_client.get_all_positions()
sers = [pd.Series(dict(x)) for x in positions]
df = pd.concat(sers, axis=1)
df.T


,asset_id,symbol,exchange,asset_class,avg_entry_price,qty,side,market_value,cost_basis,unrealized_pl,unrealized_plpc,unrealized_intraday_pl,unrealized_intraday_plpc,current_price,lastday_price,change_today
0,d1211737-a61a-434e-a3a1-fb8babdb87d6,ACT,AssetExchange.NASDAQ,AssetClass.US_EQUITY,23.88,41,PositionSide.LONG,969.65,979.08,-9.43,-0.0096314907872697,-9.43,-0.0096314907872697,23.65,23.75,-0.0042105263157895
1,f54d13d4-86f0-4ea3-8c1b-ab21e25ff3ff,AMCX,AssetExchange.NASDAQ,AssetClass.US_EQUITY,17.42,56,PositionSide.LONG,988.96,975.52,13.44,0.0137772675086108,13.44,0.0137772675086108,17.66,17.34,0.0184544405997693
2,ca73967a-2ce1-4436-a61f-eb777e7bfdfe,ARL,AssetExchange.NYSE,AssetClass.US_EQUITY,22.39,43,PositionSide.LONG,923.64,962.77,-39.13,-0.0406431442608307,-39.13,-0.0406431442608307,21.48,21.29,0.0089243776420855
3,882decf1-d315-46bd-87cf-cccb407a47ec,ARW,AssetExchange.NYSE,AssetClass.US_EQUITY,110.69,9,PositionSide.LONG,1000.89,996.21,4.68,0.0046978046797362,4.68,0.0046978046797362,111.21,110.14,0.0097149082985291
4,378b735c-c3ba-4a80-8e9d-677eb7113fd5,ATEN,AssetExchange.NYSE,AssetClass.US_EQUITY,16.0529032258064516,124,PositionSide.LONG,1960.44,1990.56,-30.12,-0.0151314203038341,-2.48,-0.0012634238787113,15.81,15.83,-0.0012634238787113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,6c8b1a1b-a455-4542-bc4d-943981e28c32,VSTO,AssetExchange.NYSE,AssetClass.US_EQUITY,26.2682142857142857,112,PositionSide.LONG,2915.36,2942.04,-26.68,-0.0090685374773966,-20.1521428571428566,-0.0068649495816865,26.03,26.18,-0.005729564553094
99,369b045b-b28f-4a27-bc52-12740675415f,VTLE,AssetExchange.NYSE,AssetClass.US_EQUITY,46.99,20,PositionSide.LONG,934.2,939.8,-5.6,-0.0059587146201319,-5.6,-0.0059587146201319,46.71,47.61,-0.0189035916824197
100,68368bd9-dbb4-4ae1-aacb-82224dd01d12,WEYS,AssetExchange.NASDAQ,AssetClass.US_EQUITY,24.08,40,PositionSide.LONG,977.6,963.2,14.4,0.0149501661129568,14.4,0.0149501661129568,24.44,23.29,0.0493774151996565
101,4b525725-a297-4fb2-94c3-7ced9a8e7bdb,WWW,AssetExchange.NYSE,AssetClass.US_EQUITY,11.48,87,PositionSide.LONG,1052.7,998.76,53.94,0.054006968641115,53.94,0.054006968641115,12.1,11.24,0.0765124555160142
